In [ ]:
import geopandas as gpd
from numpy.ma.core import append

gpd.read_file('blocks (1).geojson')

In [ ]:
# Пример вызова генплана на большой территории 

import geopandas as gpd
from shapely import Polygon, MultiPolygon

from app.gen_planner.python.src import GenPlanner, gen_plan, basic_func_zone

zones, roads = GenPlanner(gpd.read_file('1.gpkg')).poly2func2terr2block(gen_plan)

In [ ]:
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', categorical=True)
m1

In [ ]:
# тут вызовы по сценариям по разным территориям, везде basic по умолчанию, но другие лежат в zoning.


from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('Аэродром_лодейнопольское_поселение.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
# тут вызов деления территориальной зоны на кварталы по минимальному размеру терр.зоны
from app.gen_planner.python.src import GenPlanner, terr_zones
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).poly2block(terr_zones.residential_terr)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner
import geopandas as gpd

zones = {'1': 0.1,
         '2': 0.1,
         '3': 0.1,
         '4': 0.1,
         '5': 0.1,
         '6': 0.1,
         '7': 0.1,
         '8': 0.1, }
zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).split_poly(zones, roads_width=50)
zones.explore()

In [ ]:
import geopandas as gpd
import numpy as np
from shapely import MultiPolygon, Polygon

local_crs = 32636
multip = gpd.read_file('2.gpkg').to_crs(local_crs)


def elasatic_wrap(gdf: gpd.GeoDataFrame) -> Polygon:
    gdf = gdf.copy()
    multip = gpd.GeoDataFrame(geometry=gdf.geometry, crs=gdf.crs).explode(ignore_index=True)
    max_dist = np.ceil(multip.apply(lambda row: multip.drop(row.name).distance(row.geometry).min(), axis=1).max(axis=0))
    poly = multip.buffer(max_dist).union_all().buffer(-max_dist)
    if isinstance(poly, MultiPolygon):
        raise RuntimeError('Cant cast Multipolygon to Polygon')
    poly = Polygon(poly.exterior)
    return poly


p = elasatic_wrap(multip)

In [ ]:
gpd.GeoDataFrame(geometry=[p.simplify(250)], crs=local_crs).explore()

In [ ]:
from app.gen_planner.python.src import GenPlanner, gen_plan
import geopandas as gpd

gp = GenPlanner(gpd.read_file('2.gpkg'), rotation=0)
multiterritory = gp.transformed_poly
zones, roads = gp.poly2func2terr2block(gen_plan)

In [ ]:
zones.explore()

# MultiP

In [25]:
import geopandas as gpd
from app.gen_planner.python.src.utils.geom_utils import geometry_to_multilinestring
from shapely.ops import polygonize, unary_union, Polygon


def territory_splitter(gdf_to_split: gpd.GeoDataFrame, splitters: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    local_crs = gdf_to_split.estimate_utm_crs()
    gdf_to_split = gdf_to_split.to_crs(local_crs)
    splitters = splitters.to_crs(local_crs)
    lines_orig = gdf_to_split.to_crs(local_crs).geometry.apply(geometry_to_multilinestring).to_list()
    lines_splitters = splitters.to_crs(local_crs).geometry.apply(geometry_to_multilinestring).to_list()

    polygons = gpd.GeoDataFrame(geometry=list(polygonize(unary_union(lines_orig + lines_splitters))),
                                crs=local_crs).clip(gdf_to_split.to_crs(local_crs), keep_geom_type=True).explode()

    polygons_points = polygons.copy()
    polygons_points.geometry = polygons.representative_point()
    joined_ind = polygons_points.sjoin(splitters, how='inner', predicate='within').index.tolist()
    polygons['is_splitter'] = polygons.index.isin(joined_ind)
    polygons.geometry = polygons.apply(lambda x: Polygon(x.geometry.exterior) if not x['is_splitter'] else x.geometry,
                                       axis=1)
    non_splitters = polygons[~polygons['is_splitter']]
    contains = non_splitters.sjoin(non_splitters, predicate='contains')
    to_kick = contains[~(contains.index == contains['index_right'])]['index_right'].to_list()
    polygons.drop(index=to_kick, inplace=True)
    return polygons


local_crs = 32636

territory = gpd.read_file('project ШлиссельбургMulti.geojson').to_crs(local_crs)
roads = gpd.read_file('roads_shlis.geojson').to_crs(local_crs)
# roads.geometry = roads.geometry.buffer(5, resolution=2)

multipolys = territory_splitter(territory, roads)
multipolys.explore(style_kwds={'fillOpacity': 0.5})

In [26]:
from app.gen_planner.python.src import func_zones, GenPlanner

multipolys = multipolys[~multipolys['is_splitter']]
zones, roads = GenPlanner(multipolys).poly2terr2block(func_zones.residential_func_zone)

In [27]:
m1 = zones.explore(column='zone_name', tiles='CartoDB positron', cmap='Dark2', categorical=True)
multipolys.explore(m=m1, style_kwds={'fillOpacity': 0.05})


In [134]:
local_crs = 32636

lines_orig = multipolys.geometry.apply(geometry_to_multilinestring).to_list()
lines_new = zones.geometry.apply(geometry_to_multilinestring).to_list()
polygons = gpd.GeoDataFrame(geometry=list(polygonize(unary_union(lines_orig + lines_new))),
                            crs=local_crs).explode()
polygons['area'] = polygons.area
polygons_points = polygons.copy()
polygons_points.geometry = polygons_points.representative_point()
polygons_points = polygons_points.sjoin(zones, how='inner', predicate='within').drop(columns='index_right')
multipolys['orig_area'] = multipolys.area
division = multipolys.sjoin(polygons_points, how='inner', predicate='contains').drop(
    columns=['index_right', 'geometry', 'is_splitter', 'ratio'])



division['zone_str'] = division['zone_name'].apply(lambda x: x.name)
division = division.reset_index().groupby(['index', 'zone_str']).agg(
    {'area': 'sum', 'orig_area': 'first', 'zone_name': 'first'}).reset_index().set_index('index')
n_zones_in_poly = division.reset_index().groupby('index').agg({'area': lambda x: len(list(x))})
ready_zones = division.loc[n_zones_in_poly[n_zones_in_poly['area'] == 1].index].copy()
zones_to_split = division.loc[n_zones_in_poly[n_zones_in_poly['area'] != 1].index].copy()
zones_to_split['min_area'] = zones_to_split['zone_name'].apply(lambda x: x.min_block_area)


division['area']  - площадь по каждой тз в поллигоне

In [135]:
import pandas as pd

target_zones = pd.DataFrame.from_dict(func_zones.residential_func_zone.zones_ratio, orient='index',
                                      columns=['zones_ratio']).reset_index().rename(columns={'index': 'zone'})
target_zones['target_area'] = target_zones['zones_ratio'] * division.area.sum()
target_zones['min_area'] = target_zones['zone'].apply(lambda x: x.min_block_area)
target_zones['zone'] = target_zones['zone'].apply(lambda x: x.name)
target_zones.set_index('zone', inplace=True)
target_zones['area_last'] = target_zones['target_area']
target_zones

,zones_ratio,target_area,min_area,area_last
zone,,,,
residential,0.555556,3.068999e+06,160000,3.068999e+06
business,0.111111,6.137999e+05,160000,6.137999e+05
recreation,0.111111,6.137999e+05,320000,6.137999e+05
transport,0.111111,6.137999e+05,160000,6.137999e+05
agriculture,0.055556,3.068999e+05,640000,3.068999e+05
special,0.055556,3.068999e+05,160000,3.068999e+05


In [136]:
for _, row in ready_zones.iterrows():
    target_zones.at[row.zone_str, 'area_last'] = target_zones.loc[row.zone_str, 'area_last'] - row['area']
target_zones

,zones_ratio,target_area,min_area,area_last
zone,,,,
residential,0.555556,3.068999e+06,160000,2.209914e+06
business,0.111111,6.137999e+05,160000,6.137999e+05
recreation,0.111111,6.137999e+05,320000,5.960716e+05
transport,0.111111,6.137999e+05,160000,6.124948e+05
agriculture,0.055556,3.068999e+05,640000,3.068675e+05
special,0.055556,3.068999e+05,160000,3.068999e+05


In [137]:
zones_to_split['reserved_area'] = zones_to_split['area'] // zones_to_split['min_area'] * zones_to_split['min_area']

for _, row in zones_to_split.iterrows():
    target_zones.at[row.zone_str, 'area_last'] = target_zones.loc[row.zone_str, 'area_last'] - row['reserved_area']
target_zones

,zones_ratio,target_area,min_area,area_last
zone,,,,
residential,0.555556,3.068999e+06,160000,129913.564905
business,0.111111,6.137999e+05,160000,133799.890417
recreation,0.111111,6.137999e+05,320000,596071.633470
transport,0.111111,6.137999e+05,160000,292494.845771
agriculture,0.055556,3.068999e+05,640000,306867.513191
special,0.055556,3.068999e+05,160000,306899.945208


In [155]:
possible_areas = zones_to_split.reset_index().groupby('index').agg(
    {'zone_str': list, 'reserved_area': list, 'orig_area': 'first'})
possible_areas['reserved_area_sum'] = possible_areas['reserved_area'].apply(lambda x: sum(x))
possible_areas['area_last'] = possible_areas['orig_area'] -  possible_areas['reserved_area_sum'] 
possible_areas = possible_areas.join(possible_areas.apply(lambda x: pd.Series(x.reserved_area, index=x.zone_str),axis=1))
possible_areas.drop(columns=['zone_str', 'reserved_area'])

,orig_area,reserved_area_sum,area_last,agriculture,business,recreation,residential,special,transport
index,,,,,,,,,
0,2.233466e+06,2080000.0,153466.249186,0.0,NaN,0.0,2080000.0,0.0,NaN
6,3.997062e+05,0.0,399706.243180,NaN,NaN,0.0,0.0,0.0,0.0
8,5.011487e+05,160000.0,341148.749781,NaN,NaN,0.0,NaN,0.0,160000.0
11,1.923937e+03,0.0,1923.937427,NaN,NaN,0.0,NaN,NaN,0.0
18,8.097864e+04,0.0,80978.639385,NaN,NaN,0.0,0.0,NaN,NaN
44,2.204361e+05,160000.0,60436.135242,NaN,0.0,NaN,NaN,NaN,160000.0
46,8.711876e+05,480000.0,391187.627997,0.0,480000.0,NaN,0.0,0.0,0.0
47,7.418388e+04,0.0,74183.876467,NaN,NaN,NaN,0.0,0.0,NaN
48,2.425224e+04,0.0,24252.235786,NaN,NaN,NaN,0.0,0.0,NaN


In [182]:
while target_zones['area_last'].sum() > 0:
    zone_to_search = target_zones['area_last'].idxmin()
    max_area_to_add = target_zones.loc[zone_to_search, 'min_area']
    zone_to_add_id = possible_areas[~possible_areas[zone_to_search].isna()]['area_last'].idxmin()
    break

48


In [112]:
import pandas as pd

# Пример данных
data = [
    (0, ["agriculture", "residential", "special", "recreation"], [0.0, 2080000.0, 0.0, 0.0]),
    (6, ["residential", "special", "transport", "recreation"], [0.0, 0.0, 0.0, 0.0]),
    (8, ["special", "transport", "recreation"], [0.0, 160000.0, 0.0]),
    (11, ["transport", "recreation"], [0.0, 0.0]),
    (18, ["residential", "recreation"], [0.0, 0.0]),
    (44, ["business", "transport"], [0.0, 160000.0]),
    (46, ["agriculture", "residential", "business", "transport", "transport", "special"],
     [0.0, 0.0, 480000.0, 0.0, 0.0, 0.0])
]

# Создаем DataFrame
df = pd.DataFrame(data, columns=["id", "zone_str", "reserved_area"])

# Применяем apply для создания столбцов на основе зоны и значений
expanded_df = df.apply(lambda x: pd.Series(x.reserved_area, index=x.zone_str), axis=1)

# Печатаем результат
print(expanded_df)


ValueError: cannot reindex on an axis with duplicate labels

In [ ]:

from app.gen_planner.python.src import GenPlanner, basic_func_zone, func_zones
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).poly2terr(func_zones.residential_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)